# 1. Introdução

Neste laboratório será demonstrado o pre-processamento completo de uma imagem para recorte autormatico de uma área de interesse.

In [ ]:
#pacotes e funções auxiliares
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from google.colab.patches import cv2_imshow
import math

def show_image(img, title):
  plt.figure(figsize=(10,5))
  plt.imshow(img)
  plt.title(title)
  plt.axis('off')
  plt.show()


# 2. Dados

In [ ]:
# Verifica se já foram baixadas as imagens do drive, baixando-as e descompactando se necessário
! [ ! -d "/content/MAB.jpg" ] && gdown -O /content/MAB.jpg "1nb_YBLwcQVSXlliDnoByu27Oh2fzAeK8"

mab_image_path = Path("/content/MAB.jpg")

! [ ! -d "/content/black_dog.png" ] && gdown -O /content/black_dog.png "1GFXzpIGE0AO3I8AuEodf5ahE72KhyjSg"

dog_image_path = Path("/content/black_dog.png")



In [ ]:
img = cv2.imread(mab_image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

print(img.shape)
h, w, _ = img.shape

show_image(img, "Museu Aeroespacial Brasileiro")

# 3. Subtituição do Céu

## 3.1. Criação das Máscara

In [ ]:
img_original = img.copy()

#Geração de máscara binária
gray_img = cv2.cvtColor(img_original, cv2.COLOR_RGB2GRAY)
binary_img = 255 * (gray_img > 200).astype(np.uint8)
cv2_imshow(binary_img)

In [ ]:
x1 = 0
y1 = 0
x2 = w
y2 = 140

#recorte da area util da mascara
binary_cropped = binary_img[y1:y2, x1:x2]
cv2_imshow(binary_cropped)

In [ ]:
#completar a mascara com retangulo total  branco
# bottom_mask = np.full((h-y2,w), 255,dtype=np.uint8)
bottom_mask = np.zeros((h-y2,w),dtype=np.uint8)

mask = np.vstack([binary_cropped, bottom_mask])

cv2_imshow(mask)

## 3.2. Aplicar a Máscara

In [ ]:
# Criar uma nova imagem azul do tamanho da imagem original
img_sky = np.zeros_like(img_original)
img_sky[:,:,1] = 210
img_sky[:,:,2] = 255

show_image(img_sky, "Novo céu")

In [ ]:
# Recortar o céu usando a mascara
sky_filtered = cv2.bitwise_and(img_sky, img_sky, mask=mask)

show_image(sky_filtered, "Novo céu recortado")

In [ ]:
# Criar a máscara invertida
mask_inv = cv2.bitwise_not(mask)

# Selecionar a área que não é céu na imagem original
img_no_sky = cv2.bitwise_and(img_original, img_original, mask=mask_inv)

show_image(img_no_sky, "Imagem sem céu")

In [ ]:
# Recortar o novo céu usando a máscara original
sky_filtered = cv2.bitwise_and(img_sky, img_sky, mask=mask)

# Combinar a área que não é céu com o novo céu
new_image = cv2.bitwise_or(img_no_sky, sky_filtered)

show_image(new_image, "Imagem com o céu substituído")

# 4. Máscara Automático

In [ ]:
img = cv2.imread(dog_image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img_original = img.copy()

# Adiciona uma borda preta de 10 pixels em volta
img_original = cv2.copyMakeBorder(
    img_original, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=(0, 0, 0)
)


print(img.shape)
h, w, _ = img.shape

show_image(img_original, "Pronto para a call")

In [ ]:
# Converter para escala de cinza
gray = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)

# Detecção de bordas
edges = cv2.Canny(gray, threshold1=200, threshold2=650)

# Encontrar contornos nas bordas
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Desenhar os contornos
result = img_original.copy()
cv2.drawContours(result, contours, -1, (255, 0, 0), 3)

show_image(result, "Contornos")


In [ ]:
# Pega o maior contorno (assumindo que é o cachorro)
cnt = max(contours, key=cv2.contourArea)

# Criar máscara vazia
mask = np.zeros(img_original.shape[:2], dtype=np.uint8)

# Preencher o contorno na máscara
cv2.drawContours(mask, [cnt], -1, 255, thickness=-1)
# mask = cv2.bitwise_not(img_original, img_original, mask=mask)

# Aplicar a máscara na imagem
mask_green = cv2.bitwise_and(img_original, img_original, mask=mask)

cv2_imshow(mask_green)

In [ ]:
#gerar mascara binaria
gray_img = cv2.cvtColor(mask_green, cv2.COLOR_RGB2GRAY)
binary_img = 255 * (gray_img < 120).astype(np.uint8)
cv2_imshow(binary_img)


In [ ]:
#aplicar mascara binaria
final = cv2.bitwise_and(img_original, img_original, mask=binary_img)
cv2_imshow(final)
